In [10]:
from common_functions import get_secret, ret_metabase, google_sheets
from datetime import datetime, timedelta
import os
import sqlalchemy
import requests
import psycopg2
import json
from pathlib import Path
import pandas as pd
import numpy as np
import pytz
import io
from io import BytesIO


import requests
def get_jwt(country='EG'):
    import jwt
    fintech_service_account = json.loads(get_secret("prod/fintechServiceEmail/credentials"))
    fintech_service_account_emailname= fintech_service_account["email_name"]
    fintech_service_account_emailpass= fintech_service_account["email_password"]
    payload = {
            "client_id":"admin-portal",
            "grant_type":"password",
            "username":fintech_service_account_emailname,
            "password":fintech_service_account_emailpass 
        }
 
    r = requests.post("https://sso.maxab.info/auth/realms/maxab/protocol/openid-connect/token",
        headers={"Content-Type":"application/x-www-form-urlencoded"},
        data=payload
        )

    fullJwtResponse = r.json()
    jwt_access_token = fullJwtResponse['access_token']
        
    return jwt_access_token

def initialize_env():
    """Initialize environment variables if needed"""
    # Add any environment initialization logic here
    pass

def send_text_slack(channel, text):
    """
    Send a text message to a Slack channel.
    
    Args:
        channel (str): The Slack channel name to send the message to
        text (str): The message text to send
        
    Raises:
        Exception: If the message fails to send
    """
    print(f"Sending Slack message to channel '{channel}'...")
    
    import slack
    import os

    initialize_env()

    client = slack.WebClient(token=os.environ["SLACK_TOKEN"])
    try:
        client.chat_postMessage(
            channel=channel,
            text=text
        )
        print('Slack message sent successfully')
    except Exception as e:
        print(f'Failed to send Slack message: {str(e)}')
        raise e

def send_file_to_slack(channel, file_path, title=None):
    """
    Send a file to a Slack channel.
    
    Args:
        channel (str): The Slack channel name to send the file to
        file_path (str): The path to the file to upload
        title (str): Optional title for the file
        
    Raises:
        Exception: If the file upload fails
    """
    print(f"Sending file to Slack channel '{channel}': {file_path}")
    
    import slack
    import os

    initialize_env()

    client = slack.WebClient(token=os.environ["SLACK_TOKEN"])
    try:
        # Upload file to Slack
        response = client.files_upload(
            channels=channel,
            file=file_path,
            title=title or os.path.basename(file_path)
        )
        print(f'File uploaded successfully: {response["file"]["name"]}')
    except Exception as e:
        print(f'Failed to upload file to Slack: {str(e)}')
        raise e

def send_daily_report_slack(cancellation_stats, creation_stats, dispatch_stats, creation_df, execution_date):
    """
    Send a daily summary report to Slack with all task statistics and creation data as Excel file.
    
    Args:
        cancellation_stats (dict): Statistics from cancellation batch processing
        creation_stats (dict): Statistics from creation batch processing  
        dispatch_stats (dict): Statistics from dispatch batch processing
        creation_df (DataFrame): The creation data DataFrame
        execution_date (datetime): The execution date
    """
    print("Preparing daily Slack report...")
    
    # Format the execution date
    exec_date_str = execution_date.strftime("%Y-%m-%d %H:%M:%S")
    
    # Create summary statistics
    total_cancellation_batches = cancellation_stats.get('total_batches', 0)
    successful_cancellation_batches = cancellation_stats.get('successful_batches', 0)
    cancellation_success_rate = cancellation_stats.get('success_rate', 0)
    
    total_creation_batches = creation_stats.get('total_batches', 0)
    successful_creation_batches = creation_stats.get('successful_batches', 0)
    creation_success_rate = creation_stats.get('success_rate', 0)
    total_rows_created = creation_stats.get('total_rows_created', 0)
    
    total_dispatch_batches = dispatch_stats.get('total_batches', 0)
    successful_dispatch_batches = dispatch_stats.get('successful_batches', 0)
    dispatch_success_rate = dispatch_stats.get('success_rate', 0)
    total_rows_dispatched = dispatch_stats.get('total_rows_dispatched', 0)
    
    # Get creation data summary
    total_creation_rows = len(creation_df) if creation_df is not None else 0
    unique_workflows = creation_df['Workflow ID'].nunique() if creation_df is not None and 'Workflow ID' in creation_df.columns else 0
    
    # Create the Slack message
    slack_msg = f"""
:bar_chart: *Daily Ecom Dispatching Report - {exec_date_str}*

*📊 Task Processing Summary:*

*🔄 Cancellation Tasks:*
• Total Batches: {total_cancellation_batches}
• Successful: {successful_cancellation_batches}
• Success Rate: {cancellation_success_rate:.1f}%

*📝 Creation Tasks:*
• Total Batches: {total_creation_batches}
• Successful: {successful_creation_batches}
• Success Rate: {creation_success_rate:.1f}%
• Rows Created: {total_rows_created:,}

*📤 Dispatch Tasks:*
• Total Batches: {total_dispatch_batches}
• Successful: {successful_dispatch_batches}
• Success Rate: {dispatch_success_rate:.1f}%
• Rows Dispatched: {total_rows_dispatched:,}

*📈 Creation Data Summary:*
• Total Rows Processed: {total_creation_rows:,}
• Unique Workflows: {unique_workflows}

*🔍 Data Integrity Check:*
• Rows Created: {total_rows_created:,}
• Rows Dispatched: {total_rows_dispatched:,}
• Match Status: {'✅ MATCH' if total_rows_created == total_rows_dispatched else '🚨 MISMATCH'}
{f'• Difference: {abs(total_rows_created - total_rows_dispatched):,} rows' if total_rows_created != total_rows_dispatched else ''}

*🎯 Overall Performance:*
• Total Batches: {total_cancellation_batches + total_creation_batches + total_dispatch_batches}
• Total Successful: {successful_cancellation_batches + successful_creation_batches + successful_dispatch_batches}
• Overall Success Rate: {((successful_cancellation_batches + successful_creation_batches + successful_dispatch_batches) / max(1, total_cancellation_batches + total_creation_batches + total_dispatch_batches) * 100):.1f}%

:white_check_mark: Daily dispatching process completed!


    """
    
    # Send the text message first
    send_text_slack(channel='seif_error_logs', text=slack_msg)
    
    # Now send the Excel file with creation data
    if creation_df is not None and len(creation_df) > 0:
        try:
            # Create Excel file with creation data
            excel_filename = f"creation_data_{execution_date.strftime('%Y%m%d_%H%M%S')}.xlsx"
            
            # Save DataFrame to Excel file
            creation_df.to_excel(excel_filename, index=False)
            
            # Send file to Slack
            send_file_to_slack(channel='seif_error_logs', file_path=excel_filename, 
                              title=f"Creation Data - {execution_date.strftime('%Y-%m-%d')}")
            
            # Clean up the temporary file
            if os.path.exists(excel_filename):
                os.remove(excel_filename)
                
            print(f"Excel file sent to Slack: {excel_filename}")
            
        except Exception as e:
            print(f"Failed to send Excel file to Slack: {str(e)}")
            # Don't fail the entire process if file upload fails
    else:
        print("No creation data to send as Excel file")

def task_fail_slack_alert(context):
    """
    Send a Slack alert when an Airflow task fails.
    
    This function is called by Airflow's on_failure_callback and formats
    the error information into a readable Slack message.
    
    Args:
        context (dict): Airflow context containing task failure information
    """
    print("Preparing Slack failure alert...")
    
    slack_msg = """
        :red_circle: Task Failed.
        *Task*: {task}  
        *Dag*: {dag} 
        *Execution Time*: {exec_date}  
        *Reason*: {exception}
    """.format(
        task=context.get('task_instance').task_id,
        dag=context.get('task_instance').dag_id,
        exec_date=context.get('execution_date'),
        exception=context.get('exception')
    )

    send_text_slack(channel='seif_error_logs', text=slack_msg)

def EG_Ecom_on_ground_dispatching():
    """
    Main function for Egypt Ecom on ground dispatching process
    """
    print("Starting Egypt Ecom on Ground Dispatching process...")
    
    # Get data from Metabase
    df = ret_metabase("Egypt", 63605)  
    creation_df = df.copy()
    creation_df = creation_df[['RETAILER_ID', 'Workflow ID', 'Date', 'Description']]
    # google_sheets("On_Ground_Dispatch_LOGS", "fetched_data", "overwrite", df=df)

    # Separate DataFrame for cancellation with different query ID
    # df_cancellation = ret_metabase("Egypt", 62832)
    # cancellation_df = df_cancellation.copy()

    AT = get_jwt()

    # ------------------------
    # Bulk Cancellation with Batch Processing (2000 batch limit)
    # ------------------------

#     def process_cancellation_batch(batch_df, batch_number, total_batches):
#         """
#         Process a single batch of data for bulk task cancellation
#         """
#         # Create a temporary file for this batch
#         batch_file_path = f"data_for_cancellation_batch_{batch_number}.xlsx"
        
#         try:
#             # Save batch DataFrame to an Excel file
#             batch_df.to_excel(batch_file_path, index=False)
            
#             # Prepare the request
#             url = "https://api.maxab.info/logistics/task-based/api/portal/v1/tasks/sheets/cancellation"
            
#             headers = {
#                 "accept": "application/json, text/plain, */*",
#                 "authorization": f"Bearer {AT}",
#                 "country_identifier": "EG",
#                 "language": "AR",
#                 "origin": "https://logistics.maxab.info",
#                 "referer": "https://logistics.maxab.info/",
#                 "user-agent": "Mozilla/5.0",
#             }
            
#             files = {
#                 "file": (batch_file_path, open(batch_file_path, "rb"), "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")
#             }
            
#             # Send POST request with file upload
#             response = requests.post(url, headers=headers, files=files)
            
#             print(f"Cancellation Batch {batch_number}/{total_batches} - Response: {response.status_code}")
            
#             if response.status_code in [200, 201]:
#                 print(f"✓ Cancellation Batch {batch_number} processed successfully")
#             else:
#                 print(f"✗ Cancellation Batch {batch_number} failed with status {response.status_code}")
#                 print(f"Response: {response.text}")
                
#             return response.status_code in [200, 201]
            
#         except Exception as e:
#             print(f"✗ Error processing cancellation batch {batch_number}: {str(e)}")
#             return False
            
#         finally:
#             # Clean up temporary file
#             if os.path.exists(batch_file_path):
#                 os.remove(batch_file_path)

#     # Calculate batch size and number of batches for cancellation
#     CANCELLATION_BATCH_SIZE = 2000
#     total_cancellation_rows = len(cancellation_df)
#     total_cancellation_batches = (total_cancellation_rows + CANCELLATION_BATCH_SIZE - 1) // CANCELLATION_BATCH_SIZE  # Ceiling division

#     print(f"\n=== Bulk Cancellation Tasks ===")
#     print(f"Total cancellation rows to process: {total_cancellation_rows}")
#     print(f"Cancellation batch size: {CANCELLATION_BATCH_SIZE}")
#     print(f"Total cancellation batches: {total_cancellation_batches}")

#     # Process cancellation data in batches
#     successful_cancellation_batches = 0
#     failed_cancellation_batches = 0

#     for batch_num in range(1, total_cancellation_batches + 1):
#         start_idx = (batch_num - 1) * CANCELLATION_BATCH_SIZE
#         end_idx = min(batch_num * CANCELLATION_BATCH_SIZE, total_cancellation_rows)
        
#         batch_df = cancellation_df.iloc[start_idx:end_idx].copy()
        
#         print(f"\nProcessing cancellation batch {batch_num}/{total_cancellation_batches} (rows {start_idx + 1}-{end_idx})")
        
#         success = process_cancellation_batch(batch_df, batch_num, total_cancellation_batches)
        
#         if success:
#             successful_cancellation_batches += 1
#         else:
#             failed_cancellation_batches += 1

#     print(f"\n=== Cancellation Batch Processing Complete ===")
#     print(f"Successful cancellation batches: {successful_cancellation_batches}/{total_cancellation_batches}")
#     print(f"Failed cancellation batches: {failed_cancellation_batches}/{total_cancellation_batches}")
#     print(f"Cancellation success rate: {(successful_cancellation_batches/total_cancellation_batches)*100:.1f}%")
    
#     # Store cancellation statistics
#     cancellation_stats = {
#         'total_batches': total_cancellation_batches,
#         'successful_batches': successful_cancellation_batches,
#         'failed_batches': failed_cancellation_batches,
#         'success_rate': (successful_cancellation_batches/total_cancellation_batches)*100 if total_cancellation_batches > 0 else 0
#     }

#     # ------------------------
#     # Bulk create Task with Batch Processing
#     # ------------------------

#     def process_batch(batch_df, batch_number, total_batches):
#         """
#         Process a single batch of data for bulk task creation with retry logic for ongoing task errors
#         """
#         import json
#         import re
        
#         current_batch = batch_df.copy()
#         max_retries = 10  # Maximum number of retries to prevent infinite loops
#         retry_count = 0
        
#         while retry_count < max_retries and len(current_batch) > 0:
#             # Create a temporary file for this batch
#             batch_file_path = f"data_for_SF_batch_{batch_number}_retry_{retry_count}.xlsx"
            
#             try:
#                 # Save batch DataFrame to an Excel file
#                 current_batch.to_excel(batch_file_path, index=False)
                
#                 # Prepare the request
#                 url = "https://api.maxab.info/logistics/task-based/api/portal/v1/tasks/sheets/creation"
                
#                 headers = {
#                     "accept": "application/json, text/plain, */*",
#                     "authorization": f"Bearer {AT}",
#                     "country_identifier": "EG",
#                     "language": "AR",
#                     "origin": "https://logistics.maxab.info",
#                     "referer": "https://logistics.maxab.info/",
#                     "user-agent": "Mozilla/5.0",
#                 }
                
#                 files = {
#                     "file": (batch_file_path, open(batch_file_path, "rb"), "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")
#                 }
                
#                 # Send POST request with file upload
#                 response = requests.post(url, headers=headers, files=files)
                
#                 print(f"Batch {batch_number}/{total_batches} (retry {retry_count}) - Response: {response.status_code}")
                
#                 if response.status_code in [200, 201]:
#                     print(f"✓ Batch {batch_number} processed successfully after {retry_count} retries")
#                     return True
#                 elif response.status_code == 400:
#                     # Try to parse the error response
#                     try:
#                         error_data = json.loads(response.text)
#                         error_message = error_data.get('message', '')
                        
#                         # Check if it's an ongoing task error
#                         if 'RETAILER_HAS_ONGOING_TASK' in error_data.get('error_code', ''):
#                             # Extract retailer ID from the error message
#                             # Pattern: "Retailer With ID 771554 Has Ongoing Task for the same workflow"
#                             match = re.search(r'Retailer With ID (\d+) Has Ongoing Task', error_message)
#                             if match:
#                                 problematic_retailer_id = int(match.group(1))
#                                 print(f"⚠️  Removing retailer ID {problematic_retailer_id} with ongoing task")
                                
#                                 # Remove the problematic retailer from the batch
#                                 current_batch = current_batch[current_batch['Retailer ID'] != problematic_retailer_id]
                                
#                                 if len(current_batch) == 0:
#                                     print(f"✗ Batch {batch_number} failed - no valid retailers remaining")
#                                     return False
                                
#                                 retry_count += 1
#                                 print(f"🔄 Retrying batch {batch_number} with {len(current_batch)} remaining retailers")
#                                 continue
#                             else:
#                                 print(f"✗ Could not extract retailer ID from error message: {error_message}")
#                                 return False
#                         else:
#                             # Other 400 error, not related to ongoing tasks
#                             print(f"✗ Batch {batch_number} failed with 400 error: {error_message}")
#                             return False
                            
#                     except json.JSONDecodeError:
#                         print(f"✗ Could not parse error response: {response.text}")
#                         return False
#                 else:
#                     # Other error status codes
#                     print(f"✗ Batch {batch_number} failed with status {response.status_code}")
#                     print(f"Response: {response.text}")
#                     return False
                    
#             except Exception as e:
#                 print(f"✗ Error processing batch {batch_number}: {str(e)}")
#                 return False
                
#             finally:
#                 # Clean up temporary file
#                 if os.path.exists(batch_file_path):
#                     os.remove(batch_file_path)
        
#         if retry_count >= max_retries:
#             print(f"✗ Batch {batch_number} failed after {max_retries} retries")
#             return False
        
#         return True

#     # Calculate batch size and number of batches
#     BATCH_SIZE = 500
#     total_rows = len(creation_df)
#     total_batches = (total_rows + BATCH_SIZE - 1) // BATCH_SIZE  # Ceiling division

#     print(f"Total rows to process: {total_rows}")
#     print(f"Batch size: {BATCH_SIZE}")
#     print(f"Total batches: {total_batches}")

#     # Process data in batches
#     successful_batches = 0
#     failed_batches = 0
#     total_rows_created = 0  # Track actual rows created

#     for batch_num in range(1, total_batches + 1):
#         start_idx = (batch_num - 1) * BATCH_SIZE
#         end_idx = min(batch_num * BATCH_SIZE, total_rows)
        
#         batch_df = creation_df.iloc[start_idx:end_idx].copy()
#         batch_row_count = len(batch_df)  # Number of rows in this batch
        
#         print(f"\nProcessing batch {batch_num}/{total_batches} (rows {start_idx + 1}-{end_idx})")
        
#         success = process_batch(batch_df, batch_num, total_batches)
        
#         if success:
#             successful_batches += 1
#             total_rows_created += batch_row_count  # Add successful batch rows to total
#         else:
#             failed_batches += 1

#     print(f"\n=== Batch Processing Complete ===")
#     print(f"Successful batches: {successful_batches}/{total_batches}")
#     print(f"Failed batches: {failed_batches}/{total_batches}")
#     print(f"Success rate: {(successful_batches/total_batches)*100:.1f}%")
#     print(f"Total rows created: {total_rows_created}")
    
#     # Store creation statistics
#     creation_stats = {
#         'total_batches': total_batches,
#         'successful_batches': successful_batches,
#         'failed_batches': failed_batches,
#         'success_rate': (successful_batches/total_batches)*100 if total_batches > 0 else 0,
#         'total_rows_created': total_rows_created
#     }

    #--------------------------
    # Export Data
    #--------------------------

    url = "https://api.maxab.info/logistics/task-based/api/portal/v1/tasks/export"

    # Get current date in YYYY-MM-DD format
    current_date = datetime.now().strftime("%Y-%m-%d")
    querystring = {"date_from": current_date, "date_to": current_date, "status_ids": "1"}

    payload = ""
    headers = {"Authorization": f"Bearer {AT}"}

    response = requests.request("GET", url, data=payload, headers=headers, params=querystring)

    print(f"2. Export Data Response: {response.status_code}")

    response = requests.get(url, headers=headers, params=querystring)
    #--------------------------
    # Merge DataFrames
    #--------------------------

    # Load Excel directly into pandas
    export_df = pd.read_excel(BytesIO(response.content))
    export_df = export_df[export_df['Status'] == 'CREATED']

    # Merge on common columns
    merged_df = export_df.merge(
        df[['RETAILER_ID', 'AGENT_ID']],   # only keep necessary columns
        how='left',                        # keep all rows from export_df
        left_on='Retailer Id',             # column name in export_df
        right_on='RETAILER_ID'             # column name in df
    )

    # Drop duplicate 'Retailer ID' (optional, since export_df already has 'Retailer Id')
    merged_df = merged_df.drop_duplicates(subset=['RETAILER_ID', 'Task Id'])
    merged_df = merged_df.drop(columns=['RETAILER_ID'])

    merged_df = merged_df[['Task Id', 'AGENT_ID']]

    #--------------------------
    # Bulk Dispatch Tasks with Batch Processing
    #--------------------------

    def process_dispatch_batch(batch_df, batch_number, total_batches):
        """
        Process a single batch of data for bulk task dispatching with retry logic for agent not found errors
        """
        import json
        import re
        
        current_batch = batch_df.copy()
        max_retries = 10  # Maximum number of retries to prevent infinite loops
        retry_count = 0
        
        while retry_count < max_retries and len(current_batch) > 0:
            # Create a temporary file for this batch
            batch_file_path = f"data_for_BD_batch_{batch_number}_retry_{retry_count}.xlsx"
            
            try:
                # Save batch DataFrame to an Excel file
                current_batch.to_excel(batch_file_path, index=False)
                
                # Prepare the request
                url = "https://api.maxab.info/logistics/task-based/api/portal/v1/tasks/sheets/dispatching"
                
                headers = {
                    "accept": "application/json, text/plain, */*",
                    "authorization": f"Bearer {AT}",
                    "country_identifier": "EG",
                    "language": "AR",
                    "origin": "https://logistics.maxab.info",
                    "referer": "https://logistics.maxab.info/",
                    "user-agent": "Mozilla/5.0",
                }
                
                files = {
                    "file": (batch_file_path, open(batch_file_path, "rb"), "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")
                }
                
                # Send POST request with file upload
                response = requests.post(url, headers=headers, files=files)
                
                print(f"Dispatch Batch {batch_number}/{total_batches} (retry {retry_count}) - Response: {response.status_code}")
                
                if response.status_code in [200, 201]:
                    print(f"✓ Dispatch Batch {batch_number} processed successfully after {retry_count} retries")
                    return True
                elif response.status_code == 400:
                    # Try to parse the error response
                    try:
                        error_data = json.loads(response.text)
                        error_message = error_data.get('message', '')
                        error_code = error_data.get('error_code', '')
                        
                        # Check if it's an agent skill mismatch error
                        if 'AGENT_SKILL_MISMATCHING' in error_code:
                            # Extract task ID from the error message
                            match = re.search(r'Row: \d+, Task (\d+): Agent skill is not matching with task skill', error_message)
                            if match:
                                problematic_task_id = int(match.group(1))
                                print(f"⚠️  Found AGENT_SKILL_MISMATCHING error for Task ID {problematic_task_id}")
                                
                                # Find the agent_id for this task in the original merged_df
                                task_row = merged_df[merged_df['Task Id'] == problematic_task_id]
                                if not task_row.empty:
                                    problematic_agent_id = task_row['AGENT_ID'].iloc[0]
                                    print(f"⚠️  Removing agent ID {problematic_agent_id} (associated with Task ID {problematic_task_id}) - skill mismatch")
                                    
                                    # Remove all rows with this agent_id from the current batch
                                    current_batch = current_batch[current_batch['AGENT_ID'] != problematic_agent_id]
                                    
                                    if len(current_batch) == 0:
                                        print(f"✗ Dispatch Batch {batch_number} failed - no valid agents remaining")
                                        return False
                                    
                                    retry_count += 1
                                    print(f"🔄 Retrying dispatch batch {batch_number} with {len(current_batch)} remaining agents")
                                    continue
                                else:
                                    print(f"✗ Could not find Task ID {problematic_task_id} in merged_df")
                                    return False
                            else:
                                print(f"✗ Could not extract task ID from error message: {error_message}")
                                return False
                        else:
                            # Other 400 error, not related to agent skill mismatch
                            print(f"✗ Dispatch Batch {batch_number} failed with 400 error: {error_message}")
                            return False
                            
                    except json.JSONDecodeError:
                        print(f"✗ Could not parse error response: {response.text}")
                        return False
                elif response.status_code == 404:
                    # Try to parse the error response
                    try:
                        error_data = json.loads(response.text)
                        error_message = error_data.get('message', '')
                        
                        # Check if it's an agent not found error
                        if 'agent.NotFound' in error_data.get('error_code', ''):
                            # Extract agent ID and row number from the error message
                            # Pattern: "Row: 141, Agent with ID 485 is not found"
                            match = re.search(r'Row: (\d+), Agent with ID (\d+) is not found', error_message)
                            if match:
                                problematic_row = int(match.group(1))
                                problematic_agent_id = int(match.group(2))
                                print(f"⚠️  Removing agent ID {problematic_agent_id} (row {problematic_row}) - agent not found")
                                
                                # Remove the problematic agent from the batch
                                # Note: We need to remove by agent ID since that's what's causing the issue
                                current_batch = current_batch[current_batch['AGENT_ID'] != problematic_agent_id]
                                
                                if len(current_batch) == 0:
                                    print(f"✗ Dispatch Batch {batch_number} failed - no valid agents remaining")
                                    return False
                                
                                retry_count += 1
                                print(f"🔄 Retrying dispatch batch {batch_number} with {len(current_batch)} remaining agents")
                                continue
                            else:
                                print(f"✗ Could not extract agent ID from error message: {error_message}")
                                return False
                        else:
                            # Other 404 error, not related to agent not found
                            print(f"✗ Dispatch Batch {batch_number} failed with 404 error: {error_message}")
                            return False
                            
                    except json.JSONDecodeError:
                        print(f"✗ Could not parse error response: {response.text}")
                        return False
                else:
                    # Other error status codes
                    print(f"✗ Dispatch Batch {batch_number} failed with status {response.status_code}")
                    print(f"Response: {response.text}")
                    return False
                    
            except Exception as e:
                print(f"✗ Error processing dispatch batch {batch_number}: {str(e)}")
                return False
                
            finally:
                # Clean up temporary file
                if os.path.exists(batch_file_path):
                    os.remove(batch_file_path)
        
        if retry_count >= max_retries:
            print(f"✗ Dispatch Batch {batch_number} failed after {max_retries} retries")
            return False
        
        return True

    # Calculate batch size and number of batches for dispatch
    DISPATCH_BATCH_SIZE = 500
    total_dispatch_rows = len(merged_df)
    total_dispatch_batches = (total_dispatch_rows + DISPATCH_BATCH_SIZE - 1) // DISPATCH_BATCH_SIZE  # Ceiling division

    print(f"\n=== Bulk Dispatch Tasks ===")
    print(f"Total dispatch rows to process: {total_dispatch_rows}")
    print(f"Dispatch batch size: {DISPATCH_BATCH_SIZE}")
    print(f"Total dispatch batches: {total_dispatch_batches}")

    data = df[df["COUPON_VALUE"].notnull()]
    
    coupons = data[["RETAILER_ID", "COUPON_VALUE"]].rename(
    columns={
        "RETAILER_ID": "retailer_id",
        "COUPON_VALUE": "amount"})

    coupons["reason_id"] = 69
    coupons["type_id"] = 4
    coupons["redemption_method"] =''

    # Send in batches of max 500 rows per request

    import time
    JT = get_jwt()

    url = "https://api.maxab.info/commerce/api/admins/v1/wallet/transactions"
    data = {
        "compensation_coupon_rule_id": "343",
        "incentive_coupon_rule_id": "344"
    }
    headers = {
        "Authorization": f"Bearer {JT}"
    }

    BATCH_SIZE = 500
    total_rows = len(coupons)
    total_batches = (total_rows + BATCH_SIZE - 1) // BATCH_SIZE
    print(f"Uploading coupons in {total_batches} batch(es) of up to {BATCH_SIZE} rows...")

    for batch_index in range(total_batches):
        start_idx = batch_index * BATCH_SIZE
        end_idx = min((batch_index + 1) * BATCH_SIZE, total_rows)
        batch_df = coupons.iloc[start_idx:end_idx].copy()

        buf = BytesIO()
        with pd.ExcelWriter(buf, engine="xlsxwriter") as writer:
            batch_df.to_excel(writer, index=False)
        buf.seek(0)

        files = {
            "file": (f"coupons_batch_{batch_index + 1}_of_{total_batches}.xlsx", buf, "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")
        }

        print(f"Sending batch {batch_index + 1}/{total_batches} (rows {start_idx + 1}-{end_idx})")

        # Perform the POST request with simple 429 retry (sleep 3 minutes)
        max_429_retries = 4
        response = None
        for attempt in range(1, max_429_retries + 1):
            try:
                if hasattr(files.get("file", (None, None))[1], 'seek'):
                    files["file"][1].seek(0)
            except Exception:
                pass

            response = requests.post(url, headers=headers, files=files, data=data)
            if response.status_code == 429 and attempt < max_429_retries:
                print(f"HTTP 429 received for batch {batch_index + 1} (attempt {attempt}/{max_429_retries}). Sleeping 180 seconds before retry...")
                time.sleep(180)
                continue
            break

        print(f"Batch {batch_index + 1}/{total_batches} response: {response.status_code}")
        try:
            print(response.text)
        except Exception:
            pass

    print(response.status_code)
    print(response.text)



    # Process dispatch data in batches
    successful_dispatch_batches = 0
    failed_dispatch_batches = 0
    total_rows_dispatched = 0  # Track actual rows dispatched

    for batch_num in range(1, total_dispatch_batches + 1):
        start_idx = (batch_num - 1) * DISPATCH_BATCH_SIZE
        end_idx = min(batch_num * DISPATCH_BATCH_SIZE, total_dispatch_rows)
        
        batch_df = merged_df.iloc[start_idx:end_idx].copy()
        batch_row_count = len(batch_df)  # Number of rows in this batch
        
        print(f"\nProcessing dispatch batch {batch_num}/{total_dispatch_batches} (rows {start_idx + 1}-{end_idx})")
        
        success = process_dispatch_batch(batch_df, batch_num, total_dispatch_batches)
        
        if success:
            successful_dispatch_batches += 1
            total_rows_dispatched += batch_row_count  # Add successful batch rows to total
        else:
            failed_dispatch_batches += 1

    print(f"\n=== Dispatch Batch Processing Complete ===")
    print(f"Successful dispatch batches: {successful_dispatch_batches}/{total_dispatch_batches}")
    print(f"Failed dispatch batches: {failed_dispatch_batches}/{total_dispatch_batches}")
    print(f"Dispatch success rate: {(successful_dispatch_batches/total_dispatch_batches)*100:.1f}%")
    print(f"Total rows dispatched: {total_rows_dispatched}")
    
    # Store dispatch statistics
    dispatch_stats = {
        'total_batches': total_dispatch_batches,
        'successful_batches': successful_dispatch_batches,
        'failed_batches': failed_dispatch_batches,
        'success_rate': (successful_dispatch_batches/total_dispatch_batches)*100 if total_dispatch_batches > 0 else 0,
        'total_rows_dispatched': total_rows_dispatched
    }

    # Check for row count mismatch between creation and dispatch
    print(f"\n=== Row Count Verification ===")
    print(f"Total rows created: {total_rows_created}")
    print(f"Total rows dispatched: {total_rows_dispatched}")
    
    if total_rows_created != total_rows_dispatched:
        mismatch_alert = f"""
🚨 *ROW COUNT MISMATCH ALERT* 🚨

*Creation vs Dispatch Row Count Mismatch Detected!*

📊 *Row Counts:*
• Rows Created: {total_rows_created:,}  
• Rows Dispatched: {total_rows_dispatched:,}
• Difference: {abs(total_rows_created - total_rows_dispatched):,} rows
        """
        
        print("🚨 ROW COUNT MISMATCH DETECTED! 🚨")
        print(f"Created: {total_rows_created}, Dispatched: {total_rows_dispatched}")
        print("Sending mismatch alert to Slack...")
        
        try:
            send_text_slack(channel='seif_error_logs', text=mismatch_alert)
            print("Mismatch alert sent to Slack successfully!")
        except Exception as e:
            print(f"Failed to send mismatch alert to Slack: {str(e)}")
    else:
        print("✅ Row counts match - no data integrity issues detected")

    print("Egypt agent dispatching process completed successfully!")
    
    # Send daily report to Slack
    try:
        from airflow.utils.context import Context
        execution_date = datetime.now()  # Default to current time if not in Airflow context
        send_daily_report_slack(cancellation_stats, creation_stats, dispatch_stats, creation_df, execution_date)
        print("Daily report sent to Slack successfully!")
    except Exception as e:
        print(f"Failed to send daily report to Slack: {str(e)}")
        # Don't fail the entire task if Slack reporting fai

In [11]:
EG_Ecom_on_ground_dispatching()

Starting Egypt Ecom on Ground Dispatching process...
/home/ec2-user/service_account_key.json
2. Export Data Response: 200


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")



=== Bulk Dispatch Tasks ===
Total dispatch rows to process: 10000
Dispatch batch size: 500
Total dispatch batches: 20
Uploading coupons in 2 batch(es) of up to 500 rows...
Sending batch 1/2 (rows 1-500)
Batch 1/2 response: 200
PK SF[               [Content_Types].xml�S�n�0����*6�PU�C���\{�X�%����]8�R�
q�cfgfW�d�q�ZCB|��|�*�*h㻆},^�{Va�^K<4�6�NXQ�ǆ�9�!P��$��҆�d�c�D�j);��ѝP�g��E�M'O�ʕ����H7L�h���R���G��^�'�{�zސʮB��3�˙��h.�h�W�жF�j娄CQՠ똈���}ιL�U:D�����%އ����,�B���[�	�� ;˱�	�{N��~��X�p�ykOL��kN�V��ܿBZ~����q�� �ar��{O�PKz��q;    PK SF[               _rels/.rels���j�0�_���8�`�Q��2�m��4[ILb��ږ���.[K
�($}��v?�I�Q.���uӂ�h���x>=��@��p�H"�~�}�	�n����*"�H�׺؁�����8�Z�^'�#��7m{��O�3���G�u�ܓ�'��y|a�����D�	��l_EYȾ� ���vql3�ML�eh���*���\3�Y0���oJ׏�	:����}PK��z��   I  PK SF[               docProps/app.xmlM��
�0D�~EȽ��ADҔ��A? ��6�lB�J?ߜ���0���ͯ�)�@��׍H6���V>��$;�SC
�GS�b����l�&�e��L!y�%��49��`_���4G���F��J��Wg
~�

NameError: name 'total_rows_created' is not defined

In [5]:
from io import BytesIO
